In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Conv2D, Flatten, MaxPooling2D
from tensorflow.keras.preprocessing import image_dataset_from_directory, image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
cd /content/drive/MyDrive/Datasets

/content/drive/MyDrive/Datasets


In [ ]:
!unzip cat_dog_classifier.zip

Streaming output truncated to the last 5000 lines.
  inflating: val/cats/cat.10000.jpg  
  inflating: val/cats/cat.10001.jpg  
  inflating: val/cats/cat.10002.jpg  
  inflating: val/cats/cat.10003.jpg  
  inflating: val/cats/cat.10004.jpg  
  inflating: val/cats/cat.10005.jpg  
  inflating: val/cats/cat.10006.jpg  
  inflating: val/cats/cat.10007.jpg  
  inflating: val/cats/cat.10008.jpg  
  inflating: val/cats/cat.10009.jpg  
  inflating: val/cats/cat.10010.jpg  
  inflating: val/cats/cat.10011.jpg  
  inflating: val/cats/cat.10012.jpg  
  inflating: val/cats/cat.10013.jpg  
  inflating: val/cats/cat.10014.jpg  
  inflating: val/cats/cat.10015.jpg  
  inflating: val/cats/cat.10016.jpg  
  inflating: val/cats/cat.10017.jpg  
  inflating: val/cats/cat.10018.jpg  
  inflating: val/cats/cat.10019.jpg  
  inflating: val/cats/cat.10020.jpg  
  inflating: val/cats/cat.10021.jpg  
  inflating: val/cats/cat.10022.jpg  
  inflating: val/cats/cat.10023.jpg  
  inflating: val/cats/cat.10024.jpg  

In [ ]:
#define the paths

training_path = '/content/drive/MyDrive/Datasets/train'
testing_path = '/content/drive/MyDrive/Datasets/val'
prediction_image_path = '/content/dog.jpg'

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255, #All the pixel values would be 0-1
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
training_set = train_datagen.flow_from_directory(training_path,
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode="binary",)

test_set = test_datagen.flow_from_directory(testing_path,
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [ ]:
training_data = next(training_set)
label_data = next(test_set)

In [ ]:
#shape of the training data

#for image
print(training_data[0].shape)

#for label
print(training_data[1].shape)

(32, 64, 64, 3)
(32,)


In [ ]:
# define the architecture

# Initialising the CNN
classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units = 512, activation = 'relu')) 
Dropout(0.7)
classifier.add(Dense(units = 256, activation = 'relu')) 
Dropout(0.7)
classifier.add(Dense(units = 128, activation = 'relu')) 
Dropout(0.7)
classifier.add(Dense(units = 64, activation = 'relu')) 
Dropout(0.7)
classifier.add(Dense(units = 32, activation = 'relu')) 
Dropout(0.7)
#Sigmoid is used because this is a binary classification. For multiclass softmax
classifier.add(Dense(units = 1, activation = 'sigmoid'))

In [ ]:
# Compiling the CNN
#adam is for stochastic gradient descent 
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [ ]:
classifier.fit(training_set,
                         epochs = 50,
                         validation_data = test_set)


Epoch 1/50
625/625 [==============================] - 227s 362ms/step - loss: 0.6920 - accuracy: 0.5041 - val_loss: 0.6903 - val_accuracy: 0.5172
Epoch 2/50
625/625 [==============================] - 227s 363ms/step - loss: 0.6930 - accuracy: 0.5035 - val_loss: 0.6930 - val_accuracy: 0.5018
Epoch 3/50
625/625 [==============================] - 227s 363ms/step - loss: 0.6900 - accuracy: 0.5147 - val_loss: 0.6894 - val_accuracy: 0.5320
Epoch 4/50
625/625 [==============================] - 228s 365ms/step - loss: 0.6749 - accuracy: 0.5706 - val_loss: 0.6562 - val_accuracy: 0.6210
Epoch 5/50
625/625 [==============================] - 228s 364ms/step - loss: 0.6302 - accuracy: 0.6449 - val_loss: 0.5985 - val_accuracy: 0.6768
Epoch 6/50
625/625 [==============================] - 226s 362ms/step - loss: 0.5813 - accuracy: 0.6944 - val_loss: 0.5487 - val_accuracy: 0.7150
Epoch 7/50
625/625 [==============================] - 224s 358ms/step - loss: 0.5638 - accuracy: 0.7085 - val_loss: 0.5531 -

In [ ]:
classifier.save('cat_dog_prdicitorf.h5')

In [ ]:
model = load_model('/content/drive/MyDrive/Datasets/cat_dog_prdicitorf.h5')

In [ ]:
test_image = image.load_img(f'{prediction_image_path}', target_size=(64,64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
result = model.predict(test_image)

In [ ]:
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(training_set.class_indices)    
print(prediction)
print(result)